## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3,3), input_shape=(32,32,3), activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, (3,3), activation='relu'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#GlobalAveragePooling2D
# classifier.add(GlobalAveragePooling2D())

#FC
classifier.add(Dense(100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(units=10, activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Epoch 1/100
50000/50000 [==============================] - 9s 173us/step - loss: 1.3521 - acc: 0.5566
Epoch 2/100
50000/50000 [==============================] - 8s 151us/step - loss: 0.7696 - acc: 0.7325
Epoch 3/100
50000/50000 [==============================] - 8s 152us/step - loss: 0.5067 - acc: 0.8230
Epoch 4/100
50000/50000 [==============================] - 8s 156us/step - loss: 0.3187 - acc: 0.8892
Epoch 5/100
50000/50000 [==============================] - 8s 160us/step - loss: 0.2005 - acc: 0.9317
Epoch 6/100
50000/50000 [==============================] - 8s 166us/step - loss: 0.1480 - acc: 0.9493
Epoch 7/100
50000/50000 [==============================] - 8s 163us/step - loss: 0.1235 - acc: 0.9575
Epoch 8/100
50000/50000 [==============================] - 8s 154us/step - loss: 0.1090 - acc: 0.9627
Epoch 9/100
50000/50000 [==============================] - 8s 151us/step - loss: 0.0963 - acc: 0.9679
Epoch 10/100
50000/50000 [==============================] - 8s 156us/step - loss: 

50000/50000 [==============================] - 8s 152us/step - loss: 0.0186 - acc: 0.9948
Epoch 80/100
50000/50000 [==============================] - 8s 152us/step - loss: 0.0226 - acc: 0.9940
Epoch 81/100
50000/50000 [==============================] - 8s 155us/step - loss: 0.0115 - acc: 0.9964
Epoch 82/100
50000/50000 [==============================] - 8s 151us/step - loss: 0.0119 - acc: 0.9965
Epoch 83/100
50000/50000 [==============================] - 8s 152us/step - loss: 0.0093 - acc: 0.9972
Epoch 84/100
50000/50000 [==============================] - 8s 165us/step - loss: 0.0095 - acc: 0.9973
Epoch 85/100
50000/50000 [==============================] - 8s 153us/step - loss: 0.0153 - acc: 0.9956
Epoch 86/100
50000/50000 [==============================] - 8s 157us/step - loss: 0.0121 - acc: 0.9963
Epoch 87/100
50000/50000 [==============================] - 8s 156us/step - loss: 0.0142 - acc: 0.9962
Epoch 88/100
50000/50000 [==============================] - 8s 160us/step - loss: 0.01

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [12]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[7.0731280e-06, 5.0197828e-09, 6.5415583e-07, 9.9999177e-01,
        7.3678869e-11, 4.3062880e-12, 5.3700190e-11, 8.3931245e-10,
        5.2411730e-07, 2.2708951e-16]], dtype=float32)

In [13]:
loss, accuracy = classifier.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 112us/step


In [14]:
print("loss:{} / accuracy:{}".format(loss, accuracy*100))

loss:3.364663809585571 / accuracy:67.67999999999999
